In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
import skimage.morphology as morphology
from skimage import filters
from skimage.filters import threshold_niblack
from skimage import measure
from scipy.signal import convolve2d
import matplotlib.pyplot as plot

"""
    Features:16

    From the histogram of radios:
        mu_radios, var_radios, RBC_obj, platelets_obj
    From the histogram of areas:
        mu_2,var_2,mu_3,var_3,RBC,platelets
    Actually in pipeline:
        entropy,var_laplacian
    Others:
        measureEntropie,varLaplace,GDER,measureVariance
"""

In [ ]:
def Histogram_radios(image,debug=False):
    
    """
    Parameters
    ----------
    img: 2D array
        Input image
    debug : bool, optional
        turn on debugging?. The default is False.

    Returns
    -------
    mu_radio : float
        The mean radio of the circles inscrite over the RBC-objects.
    var_radio : float
        The var radio of the circles inscrite over the RBC-objects.
    objects: Number of circles detected as representation of the RBC count
    platelets: int
    """
    image= cv2.medianBlur(image, 5)
    im_g =image[:,:,0]

    circles = cv2.HoughCircles(im_g.astype('uint8'), cv2.HOUGH_GRADIENT, .8, 15,
                                param1=30, param2=10, minRadius=2, maxRadius=20)
    if circles is None:
        circles=np.array([[[10,10,10]]])
    radios =(circles [0,:,2])
    plt_radios=0
    for r in radios:
        if r<6:
            plt_radios=plt_radios+1  
    RBC_radios=len(circles[0])
    mu_radios=np.mean(radios) #Calcula la media 
    var_radios=np.var(radios)
        
    #plot the histogram
    if debug:
        draw_circles = np.uint16(np.around(circles))
        for i in draw_circles[0,:]:
            # dibujar circulo 
            cv2.circle(image, (i[0], i[1]), i[2], (0,255,0), 2)
            # dibujar centro
            cv2.circle(image, (i[0], i[1]), 2, (0,0,255), 1)
        fig=plot.figure(figsize=(12, 12))
        fig.add_subplot(2,2,1)
        plot.title("Original image")
        plot.imshow(image)
        fig.add_subplot(2,2,2)
        plot.hist(x=radios, bins=30, color='#F2AB6D', rwidth=0.85)
        plot.title('Radios')
        plot.xlabel('Magnitud')
        plot.ylabel('Frecuency')
        
    return mu_radios, var_radios, RBC_radios, plt_radios

In [ ]:
def Histogram_areas(image,debug=False):
    """
    Parameters
    ----------
    img: 2D array
        Input image
    debug : bool, optional
        turn on debugging?. The default is False.

    Returns
    -------
    
    Sometimes the shape of the histogram reveal a bad segmentation.If the histogram
    contains a lot of big objects produced by the segmentation process we can fix
    eliminating big areas bigger than (mu_2+sigma_2)
    
    mu_2 : float
        The mean area of the histogram of areas.
    var_2 : float
        The var area of the histogram of areas.
    mu_3 : float
        The mean area of the fixed histogram of areas.
    var_3 : float
        The var area of the fixed histogram of areas.
    Platelets : float
        Objects of a small size detected isolated
    """
    x1=0
    x2=image.shape[0]
    x3=0
    x4=image.shape[1]
    image= cv2.medianBlur(image,9)
    window_size = 135
    im_g=image[:,:,0]
    thresh_niblack = threshold_niblack(im_g, window_size=window_size, k=1)
    bw = im_g < thresh_niblack
    bw2 = morphology.remove_small_objects(bw, min_size= 5).astype('uint8')
    bw2 = morphology.remove_small_holes(bw2).astype('uint8') 

    bw3 = cv2.rectangle(
        img = bw2,
        pt1 = (0,0),
        pt2 = (abs(x4-x3),abs(x2-x1)), 
        color = 1, 
        thickness = 5
    ).astype('uint8') 
    
    regions = measure.regionprops(morphology.label(bw3))
    c=[]
    plt_areas=0
    for props in regions:
        r=props.area
        c.append(r)
        if r<90 and r>1:
            plt_areas=plt_areas+1  
    c.pop(c.index(max(c))) #Eliminate the biggest element generated by rectangle
    mu_1=350
    mu_2 = np.mean(c)
    sigma_2 =np.std(c,ddof=0) 
    #print(sigma_2)
    var_2= np.var(c)
    RBC_areas=(sum(c))/350
    d=[ x for x in c if x<500]
    mu_3=np.mean(d)
    var_3= np.var(d)
    if debug:
        fig=plot.figure(figsize=(12, 12))
        fig.add_subplot(2,3,1)
        plot.title("Original image")
        plot.imshow(image)
        fig.add_subplot(2,3,2)
        plot.title("Segmentation")
        plot.imshow(bw3)
        fig.add_subplot(2,3,3)
        plot.hist(x=c, bins=30, color='#F2AB6D', rwidth=0.85)
        plot.title('Areas')
        plot.xlabel('Size')
        plot.ylabel('Frecuency')
        plot.axvline(x = 500, color = 'b') 
  
    return mu_2,var_2,mu_3,var_3,RBC_areas,plt_areas


In [ ]:
#functions currently in the pipeline
def entropy(labels, base=None):
    value,counts = np.unique(labels, return_counts=True)
    norm_counts = counts / counts.sum()
    base = 2 if base is None else base
    return -(norm_counts * np.log(norm_counts)/np.log(base)).sum()


def var_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [ ]:
def GDER(image, wsize=3):
    """
    Gaussian derivative

    Parameters
    ----------
    img: 2D array.
        input image.
    wsize : int
        Size of kernel, must be odd. Default is 3.

    Returns
    -------
    GDER : float
        Focus metric (Gaussian derivative) of the given image.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    N = np.floor(wsize/2)
    sig = N/2.5
    [x, y] = np.meshgrid(np.arange(-N, N+1), np.arange(-N, N+1))
    G = np.exp(-(x**2 + y**2) / (2 * sig**2) / (2 * np.pi * sig))
    Gx = -x * G / (sig**2)
    Gx = Gx / sum(abs(Gx.flatten()))
    Gy = -y * G / (sig**2)
    Gy = Gy / sum(abs(Gy.flatten()))

    Rx = convolve2d(gray, Gx, mode='valid')
    Ry = convolve2d(gray, Gy, mode='valid')

    FM = Rx**2 + Ry**2

    GDER = np.mean(FM)
    return GDER


In [ ]:
if (0<(mu_2_u-mu_3_u)<230 
    and plt_radios<150 
    and plt_areas<270
    and entropy<5.3 
    and var_laplacian<40 
    and GDER<40 
    and RBC_radios<1600):